In [ ]:
if __name__ == "__main__":
    """Quick manual test for fetching a transcript."""
    video_id = "J8lxHyQmcaI"  # sample video id
    # fetcher = TranscriptFetcher()
    # transcript = fetcher.fetch_transcript(video_id)
    # if transcript:
    #     print(json.dumps({"video_id": video_id, "excerpt": transcript[:500]}, indent=2))
    # else:
    #     sys.exit(1)

    from youtube_transcript_api import YouTubeTranscriptApi

    ytt_api = YouTubeTranscriptApi()
    ytt_api.fetch(video_id, languages = ["en"])

In [ ]:
if __name__ == "__main__":
    """Quick manual test for fetching a transcript."""
    video_id = "J8lxHyQmcaI"  # sample video id
    # fetcher = TranscriptFetcher()
    # transcript = fetcher.fetch_transcript(video_id)
    # if transcript:
    #     print(json.dumps({"video_id": video_id, "excerpt": transcript[:500]}, indent=2))
    # else:
    #     sys.exit(1)

    from youtube_transcript_api import YouTubeTranscriptApi

    ytt_api = YouTubeTranscriptApi()
    fetched_transcript = ytt_api.fetch(video_id, languages = ["en"])
    print(fetched_transcript[0])

FetchedTranscriptSnippet(text='Did you know that Nvidia is the most', start=0.0, duration=3.36)


In [ ]:
fetched_transcript

FetchedTranscript(snippets=[FetchedTranscriptSnippet(text='Did you know that Nvidia is the most', start=0.0, duration=3.36), FetchedTranscriptSnippet(text='valuable company in the world? I mean,', start=1.36, duration=3.919), FetchedTranscriptSnippet(text='it makes sense, right? All of these huge', start=3.36, duration=3.6), FetchedTranscriptSnippet(text='AI companies are relying on them fully', start=5.279, duration=3.761), FetchedTranscriptSnippet(text='in order to do the AI stuff that we all', start=6.96, duration=3.84), FetchedTranscriptSnippet(text="expect them to do. That's why Nvidia's", start=9.04, duration=4.24), FetchedTranscriptSnippet(text='value is comparable to that of silver.', start=10.8, duration=4.64), FetchedTranscriptSnippet(text='Yes, all of the silver in the world is', start=13.28, duration=4.32), FetchedTranscriptSnippet(text='comparable to the value of Nvidia, just', start=15.44, duration=4.32), FetchedTranscriptSnippet(text='one company that makes chips for gam

In [ ]:
fetched_transcript[1]

FetchedTranscriptSnippet(text='valuable company in the world? I mean,', start=1.36, duration=3.919)

In [ ]:
from typing import Dict, Any
import re
import json
import sys

try:
    from youtube_transcript_api import YouTubeTranscriptApi

    TRANSCRIPT_API_AVAILABLE = True
except ImportError:
    TRANSCRIPT_API_AVAILABLE = False


class TranscriptFetcher:
    """Fetches and cleans YouTube video transcripts."""

    # Common filler words/sounds to remove
    FILLER_WORDS = {
        "um",
        "uh",
        "like",
        "you know",
        "basically",
        "literally",
        "actually",
        "honestly",
        "right",
        "so",
        "yeah",
        "okay",
        "alright",
        "well",
    }

    def __init__(self):
        """Initialize TranscriptFetcher."""
        if not TRANSCRIPT_API_AVAILABLE:
            raise RuntimeError(
                "youtube-transcript-api not installed. Install with: pip install youtube-transcript-api"
            )

    def fetch_transcript(self, video_id: str, languages: list = None) -> str | None:
        """
        Fetch transcript for a YouTube video.

        Args:
            video_id: YouTube video ID
            languages: List of language codes to try (e.g., ['en', 'es'])

        Returns:
            Transcript text or None if not available
        """
        if not TRANSCRIPT_API_AVAILABLE:
            raise RuntimeError("youtube-transcript-api not installed")

        if languages is None:
            languages = ["en"]

        try:
            
            # transcript = YouTubeTranscriptApi.fetch(video_id, languages=languages)
            ytt_api = YouTubeTranscriptApi()
            transcript = ytt_api.fetch(video_id, languages=languages)
            # Merge transcript entries into continuous text
            text = " ".join([entry["text"] for entry in transcript])
            return text

        except Exception as e:
            print(f"Could not fetch transcript for video {video_id}: {e}")
            return None

    def clean_transcript(self, text: str) -> str:
        """
        Clean raw transcript text.

        Operations:
        - Remove timestamps and brackets
        - Normalize spaces
        - Remove filler words at start of sentences
        - Remove repeated punctuation

        Args:
            text: Raw transcript text

        Returns:
            Cleaned text
        """
        if not text:
            return ""

        # Remove bracketed content (like [Music], [Applause])
        text = re.sub(r"\[.*?\]", "", text)

        # Remove multiple spaces
        text = re.sub(r"\s+", " ", text)

        # Remove repeated punctuation
        text = re.sub(r"\.{2,}", ".", text)
        text = re.sub(r"!{2,}", "!", text)
        text = re.sub(r"\?{2,}", "?", text)

        # Clean up filler words at the beginning of sentences
        text = self._remove_filler_words(text)

        # Normalize spacing around punctuation
        text = re.sub(r"\s+([.,!?;:])", r"\1", text)
        text = re.sub(r"([.,!?;:])\s*([a-zA-Z])", r"\1 \2", text)

        return text.strip()

    def _remove_filler_words(self, text: str) -> str:
        """Remove filler words at the beginning of sentences."""
        sentences = text.split(". ")
        cleaned_sentences = []

        for sentence in sentences:
            words = sentence.split()
            # Remove leading filler words
            while words and words[0].lower().rstrip(",") in self.FILLER_WORDS:
                words.pop(0)

            if words:
                cleaned_sentences.append(" ".join(words))

        return ". ".join(cleaned_sentences)

    def fetch_and_clean(
        self, video_id: str, title: str, link: str, languages: list = None
    ) -> Dict[str, Any] | None:
        """
        Fetch and clean transcript in one call.

        Args:
            video_id: YouTube video ID
            title: Video title
            link: Video link
            languages: Language preferences

        Returns:
            Dict with video_id, title, clean_text, link or None
        """
        raw_text = self.fetch_transcript(video_id, languages)

        if raw_text is None:
            return None

        clean_text = self.clean_transcript(raw_text)

        return {
            "video_id": video_id,
            "title": title,
            "clean_text": clean_text,
            "link": link,
        }

In [ ]:
if __name__ == "__main__":
    """Quick manual test for fetching a transcript."""
    video_id = "J8lxHyQmcaI"  # sample video id
    fetcher = TranscriptFetcher()
    transcript = fetcher.fetch_transcript(video_id)
    if transcript:
        print(json.dumps({"video_id": video_id, "excerpt": transcript[:500]}, indent=2))
    else:
        sys.exit(1)

Could not fetch transcript for video J8lxHyQmcaI: 'FetchedTranscriptSnippet' object is not subscriptable


SystemExit: 1

c:\Users\Akaash\Desktop\D_Drive\Project\AI_NewsLetter\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3709: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
ytt_api = YouTubeTranscriptApi()
transcript = ytt_api.fetch(video_id, languages=languages)
# Merge transcript entries into continuous text
text = " ".join([entry["text"] for entry in transcript])

NameError: name 'languages' is not defined

In [ ]:
languages = ["en"]

In [ ]:
ytt_api = YouTubeTranscriptApi()
transcript = ytt_api.fetch(video_id, languages=languages)
# Merge transcript entries into continuous text
text = " ".join([entry["text"] for entry in transcript])
return text

TypeError: 'FetchedTranscriptSnippet' object is not subscriptable

In [ ]:
transcript

FetchedTranscript(snippets=[FetchedTranscriptSnippet(text='Did you know that Nvidia is the most', start=0.0, duration=3.36), FetchedTranscriptSnippet(text='valuable company in the world? I mean,', start=1.36, duration=3.919), FetchedTranscriptSnippet(text='it makes sense, right? All of these huge', start=3.36, duration=3.6), FetchedTranscriptSnippet(text='AI companies are relying on them fully', start=5.279, duration=3.761), FetchedTranscriptSnippet(text='in order to do the AI stuff that we all', start=6.96, duration=3.84), FetchedTranscriptSnippet(text="expect them to do. That's why Nvidia's", start=9.04, duration=4.24), FetchedTranscriptSnippet(text='value is comparable to that of silver.', start=10.8, duration=4.64), FetchedTranscriptSnippet(text='Yes, all of the silver in the world is', start=13.28, duration=4.32), FetchedTranscriptSnippet(text='comparable to the value of Nvidia, just', start=15.44, duration=4.32), FetchedTranscriptSnippet(text='one company that makes chips for gam

In [ ]:
text = " ".join([entry.text for entry in transcript])

In [ ]:
text

'Did you know that Nvidia is the most valuable company in the world? I mean, it makes sense, right? All of these huge AI companies are relying on them fully in order to do the AI stuff that we all expect them to do. That\'s why Nvidia\'s value is comparable to that of silver. Yes, all of the silver in the world is comparable to the value of Nvidia, just one company that makes chips for gamers. And now those same chips turn out to be really good for AI stuff, which is why everybody is using them. So, does Anthropic, oh, Anthropic\'s moving to Google\'s TPUs? Well, that\'s fine. There\'s plenty of other companies that aren\'t doing that like OpenAI. Oh, OpenAI is partnering with Cerebrris, the chip company. Well, at least there\'s Meta, right? Oh, Google\'s working to erode Nvidia\'s software advantage with Meta\'s help. Google\'s going to be giving TPUs to Meta. Huh. Very interesting. In fact, things have gotten so crazy that even Nvidia is investing in alternatives to Nvidia, like Groc

In [ ]:
len(text)

20784

In [ ]:
if __name__ == "__main__":
    """Quick manual test for fetching a transcript."""
    video_id = "J8lxHyQmcaI"  # sample video id
    fetcher = TranscriptFetcher()
    transcript = fetcher.fetch_transcript(video_id)
    if transcript:
        print(json.dumps({"video_id": video_id, "excerpt": transcript[:500]}, indent=2))
    else:
        sys.exit(1)

Could not fetch transcript for video J8lxHyQmcaI: 'FetchedTranscriptSnippet' object is not subscriptable


SystemExit: 1

c:\Users\Akaash\Desktop\D_Drive\Project\AI_NewsLetter\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3709: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from typing import Dict, Any
import re
import json
import sys

try:
    from youtube_transcript_api import YouTubeTranscriptApi

    TRANSCRIPT_API_AVAILABLE = True
except ImportError:
    TRANSCRIPT_API_AVAILABLE = False


class TranscriptFetcher:
    """Fetches and cleans YouTube video transcripts."""

    # Common filler words/sounds to remove
    FILLER_WORDS = {
        "um",
        "uh",
        "like",
        "you know",
        "basically",
        "literally",
        "actually",
        "honestly",
        "right",
        "so",
        "yeah",
        "okay",
        "alright",
        "well",
    }

    def __init__(self):
        """Initialize TranscriptFetcher."""
        if not TRANSCRIPT_API_AVAILABLE:
            raise RuntimeError(
                "youtube-transcript-api not installed. Install with: pip install youtube-transcript-api"
            )

    def fetch_transcript(self, video_id: str, languages: list = None) -> str | None:
        """
        Fetch transcript for a YouTube video.

        Args:
            video_id: YouTube video ID
            languages: List of language codes to try (e.g., ['en', 'es'])

        Returns:
            Transcript text or None if not available
        """
        if not TRANSCRIPT_API_AVAILABLE:
            raise RuntimeError("youtube-transcript-api not installed")

        if languages is None:
            languages = ["en"]

        try:
            
            # transcript = YouTubeTranscriptApi.fetch(video_id, languages=languages)
            ytt_api = YouTubeTranscriptApi()
            transcript = ytt_api.fetch(video_id, languages=languages)
            # Merge transcript entries into continuous text
            text = " ".join([entry.text for entry in transcript])
            return text

        except Exception as e:
            print(f"Could not fetch transcript for video {video_id}: {e}")
            return None

    def clean_transcript(self, text: str) -> str:
        """
        Clean raw transcript text.

        Operations:
        - Remove timestamps and brackets
        - Normalize spaces
        - Remove filler words at start of sentences
        - Remove repeated punctuation

        Args:
            text: Raw transcript text

        Returns:
            Cleaned text
        """
        if not text:
            return ""

        # Remove bracketed content (like [Music], [Applause])
        text = re.sub(r"\[.*?\]", "", text)

        # Remove multiple spaces
        text = re.sub(r"\s+", " ", text)

        # Remove repeated punctuation
        text = re.sub(r"\.{2,}", ".", text)
        text = re.sub(r"!{2,}", "!", text)
        text = re.sub(r"\?{2,}", "?", text)

        # Clean up filler words at the beginning of sentences
        text = self._remove_filler_words(text)

        # Normalize spacing around punctuation
        text = re.sub(r"\s+([.,!?;:])", r"\1", text)
        text = re.sub(r"([.,!?;:])\s*([a-zA-Z])", r"\1 \2", text)

        return text.strip()

    def _remove_filler_words(self, text: str) -> str:
        """Remove filler words at the beginning of sentences."""
        sentences = text.split(". ")
        cleaned_sentences = []

        for sentence in sentences:
            words = sentence.split()
            # Remove leading filler words
            while words and words[0].lower().rstrip(",") in self.FILLER_WORDS:
                words.pop(0)

            if words:
                cleaned_sentences.append(" ".join(words))

        return ". ".join(cleaned_sentences)

    def fetch_and_clean(
        self, video_id: str, title: str, link: str, languages: list = None
    ) -> Dict[str, Any] | None:
        """
        Fetch and clean transcript in one call.

        Args:
            video_id: YouTube video ID
            title: Video title
            link: Video link
            languages: Language preferences

        Returns:
            Dict with video_id, title, clean_text, link or None
        """
        raw_text = self.fetch_transcript(video_id, languages)

        if raw_text is None:
            return None

        clean_text = self.clean_transcript(raw_text)

        return {
            "video_id": video_id,
            "title": title,
            "clean_text": clean_text,
            "link": link,
        }

In [ ]:
if __name__ == "__main__":
    """Quick manual test for fetching a transcript."""
    video_id = "J8lxHyQmcaI"  # sample video id
    fetcher = TranscriptFetcher()
    transcript = fetcher.fetch_transcript(video_id)
    if transcript:
        print(json.dumps({"video_id": video_id, "excerpt": transcript[:500]}, indent=2))
    else:
        sys.exit(1)

{
  "video_id": "J8lxHyQmcaI",
  "excerpt": "Did you know that Nvidia is the most valuable company in the world? I mean, it makes sense, right? All of these huge AI companies are relying on them fully in order to do the AI stuff that we all expect them to do. That's why Nvidia's value is comparable to that of silver. Yes, all of the silver in the world is comparable to the value of Nvidia, just one company that makes chips for gamers. And now those same chips turn out to be really good for AI stuff, which is why everybody is using them. So"
}


In [ ]:
import requests
import re


class YouTubeChannelResolver:
    # This matches ONLY the owner channel id
    CHANNEL_ID_REGEX = re.compile(r'"externalId":"(UC[a-zA-Z0-9_-]{22})"')

    def __init__(self, timeout=10):
        self.timeout = timeout

    def get_channel_id(self, channel_url: str) -> str:
        """
        Takes any YouTube channel URL and returns permanent owner channel_id (UC...).
        Raises ValueError if not found.
        """

        # Normalize URL
        if not channel_url.startswith("http"):
            channel_url = "https://" + channel_url

        headers = {
            "User-Agent": "Mozilla/5.0"  # avoid bot blocking
        }

        response = requests.get(channel_url, headers=headers, timeout=self.timeout)
        response.raise_for_status()

        html = response.text

        match = self.CHANNEL_ID_REGEX.search(html)
        if not match:
            raise ValueError("Owner channel ID not found in page source")

        return match.group(1)


if __name__ == "__main__":
    resolver = YouTubeChannelResolver()

    test_urls = [
        "https://www.youtube.com/@t3dotgg",
        "https://www.youtube.com/@theothrowaways"
    ]

    for url in test_urls:
        try:
            channel_id = resolver.get_channel_id(url)
            print(f"URL: {url} -> Channel ID: {channel_id}")
        except Exception as e:
            print(f"Failed to resolve {url}: {e}")

URL: https://www.youtube.com/@t3dotgg -> Channel ID: UCbRP3c757lWg9M-U7TyEkXA
URL: https://www.youtube.com/@theothrowaways -> Channel ID: UCuJ0a6mGprfuF0CTp04rGjQ
